In [17]:
# https://www.kaggle.com/iafoss/image-preprocessing-128x128
import cv2
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

from sys import getsizeof, getrefcount
import psutil
import gc

In [39]:
HEIGHT = 137
WIDTH = 236
SIZE = 224

def bbox(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    return rmin, rmax, cmin, cmax

def crop_resize(img0, size=SIZE, pad=16):
    #crop a box around pixels large than the threshold 
    #some images contain line at the sides
    ymin,ymax,xmin,xmax = bbox(img0[5:-5,5:-5] > 80)
    #cropping may cut too much, so we need to add it back
    xmin = xmin - 13 if (xmin > 13) else 0
    ymin = ymin - 10 if (ymin > 10) else 0
    xmax = xmax + 13 if (xmax < WIDTH - 13) else WIDTH
    ymax = ymax + 10 if (ymax < HEIGHT - 10) else HEIGHT
    img0 = img0[ymin:ymax,xmin:xmax]
    #remove lo intensity pixels as noise
    img0[img0 < 28] = 0
    lx, ly = xmax-xmin,ymax-ymin
    l = max(lx,ly) + pad
    #make sure that the aspect ratio is kept in rescaling
    img0 = np.pad(img0, [((l-ly)//2,), ((l-lx)//2,)], mode='constant')
    return cv2.resize(img0,(size,size))

def proces_image(images):
    """
    images: (batch_size, 32332), np array
    
    """
    processed_image_list = []

    for idx in tqdm(range(len(images))):
        img0 = 255 - images[idx].reshape(HEIGHT, WIDTH).astype(np.uint8)
        #normalize each image by its max val
        img = (img0*(255.0/img0.max())).astype(np.uint8)
        processed_image_list.append(crop_resize(img))
    
    return np.array(processed_image_list)

In [40]:
from pathlib import Path
PROCESS_DATA = Path("../data/processed_data/size_224")
RAW_DATA = Path("../data")

PROCESS_DATA.mkdir(exist_ok=True, parents=True)
RAW_DATA.mkdir(exist_ok=True,parents=True)

In [ ]:
train_df = pd.read_csv("../data/train.csv")

for i in range(4):
    print(f"==================== {i} ==================")
    fn = f"train_image_data_{i}.parquet"
    process_fn = f"train_data_{i}.pickle"
    df = pd.read_parquet(RAW_DATA/fn)
    
    
    if (PROCESS_DATA/process_fn).is_file():
        print(f'parquet {i} processed already, skipped')
        continue
    
    
    
    merged_df = df.merge(train_df, on="image_id")

    image_name = merged_df["image_id"]
    label = merged_df[["grapheme_root","vowel_diacritic","consonant_diacritic"]].astype(np.uint8)
    image = merged_df.drop(["image_id", "grapheme_root","vowel_diacritic","consonant_diacritic", "grapheme"], axis=1).values

    
    # full image processing
    image = image * 1

    print(getsizeof(image)/1024/1024/1024)
    print(psutil.virtual_memory()[4]/1024/1024/1024)

    image = proces_image(image)
    print(gc.collect())
    print(getsizeof(image)/1024/1024/1024)
    print(psutil.virtual_memory()[4]/1024/1024/1024)
    
    with open(PROCESS_DATA/process_fn, "wb") as f:
#         pickle.dump(image, f)
        pickle.dump((image, image_name, label.values), f)


==================== 0 ==================


  0%|          | 141/50210 [00:00<00:35, 1405.77it/s]

1.5118995979428291
76.77163314819336


100%|██████████| 50210/50210 [00:35<00:00, 1432.89it/s]


97030
2.3463155031204224
74.90362548828125
==================== 1 ==================


...

100%|██████████| 50210/50210 [00:33<00:00, 1476.90it/s]


32350
2.3463155031204224
71.30728912353516
==================== 2 ==================


  0%|          | 144/50210 [00:00<00:34, 1432.54it/s]

1.5118995979428291
69.72334671020508


 95%|█████████▍| 47519/50210 [00:31<00:01, 1500.06it/s]

# Step by step

In [ ]:
# strange thing in memory
print(getsizeof(image)/1024/1024/1024)
print(psutil.virtual_memory()[4]/1024/1024/1024)
image = image * 1
print(getsizeof(image)/1024/1024/1024)
print(psutil.virtual_memory()[4]/1024/1024/1024)

In [ ]:
print(getsizeof(image)/1024/1024/1024)

image_max = image.max(axis=1).reshape(-1, 1).astype(np.uint8)
print(getsizeof(image_max))

image = (255 - image)/image_max*255
print(getsizeof(image)/1024/1024/1024)

image = image.reshape(-1, HEIGHT, WIDTH).astype(np.uint8)
print(getsizeof(image)/1024/1024/1024)

In [ ]:
processed_image_list = []

for idx in tqdm(range(len(image))):
    processed_image_list.append(crop_resize(image[idx]))
    

# result_list = []

# for idx in tqdm(range(len(image))):
#     root, vowel, consonant = label.values[idx]
#     result_list.append({
#         "image": crop_resize(image[idx]),
#         "name": image_name[idx],
#         "grapheme_root": root,
#         "vowel_diacritic": vowel,
#         "consonant_diacritic": consonant
#     })

In [ ]:
np.array(processed_image_list).shape

In [ ]:
# size 224 X 224 
for i in range(5):
    plt.imshow(processed_image_list[i])
    plt.show()

In [ ]:
a = (np.array(processed_image_list), image_name, label.values)

In [ ]:
a[2]

In [ ]:
with open("train_data_0.pickle", "wb") as f:
    pickle.dump(a, f)

In [15]:
with open("../data/processed_data/train_data_0.pickle", "rb") as f:
    data_list = pickle.load(f)